In [1]:
from huggingface_hub import hf_hub_download
import torch
from braindecode import EEGClassifier
from braindecode.models import EEGNetv4
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from moabb.paradigms import MotorImagery
from moabb.datasets import Zhou2016
from braindecode import EEGClassifier
from braindecode.models import EEGNetv4

from collections import OrderedDict
from torch import nn
from skorch import NeuralNet
from skorch.utils import to_numpy
from sklearn.base import TransformerMixin
from braindecode.models import EEGNetv4
from huggingface_hub import hf_hub_download
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import FunctionTransformer
from moabb.paradigms import MotorImagery
from moabb.datasets import Zhou2016
from moabb.evaluations import WithinSessionEvaluation, CrossSessionEvaluation
from sklearn.metrics import make_scorer, accuracy_score, f1_score

import pickle

import torch
import pandas as pd


/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from huggingface_hub import hf_hub_download

file_names = dict(
    torch='torch_params.pkl',
    f_params='skorch_params.pkl',
    f_optimizer='skorch_opt.pkl',
    f_history='skorch_history.json',
)
local_paths = {
    k: hf_hub_download(
        repo_id='PierreGtch/EEGNetv4',
        filename='toy/' + name,
    )
    for k, name in file_names.items()
}

In [27]:
import torch
from braindecode import EEGClassifier
from braindecode.models import EEGNetv4

# load the pure pytorch module:
torch_module = EEGNetv4(in_chans=3, n_classes=2, input_window_samples=200)
torch_module.load_state_dict(torch.load(local_paths['torch']))

# load the pure pytorch module:
skorch_module = EEGNetv4(in_chans=3, n_classes=2, input_window_samples=200)
skorch_classifier = EEGClassifier(skorch_module, max_epochs=5)
skorch_classifier.initialize()
skorch_classifier.load_params(
    f_params=local_paths['f_params'],
    f_optimizer=local_paths['f_optimizer'],
    f_history=local_paths['f_history'],
)

/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'in_chans' is depreciated. Use 'n_chans' instead.
  warnings.warn(
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'n_classes' is depreciated. Use 'n_outputs' instead.
  warnings.warn(
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/braindecode/models/base.py:23: UserWarning: EEGNetv4: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(


In [ ]:
paradigm = MotorImagery(
    channels=['C3', 'Cz', 'C4'],  # Same as the ones used to pre-train the embedding
    events=['left_hand', 'right_hand', 'feet'],
    n_classes=3,
    fmin=0.5,
    fmax=40,
    tmin=0,
    tmax=3,
    resample=128
)
datasets = [Zhou2016()]
evaluation = WithinSessionEvaluation(
    paradigm=paradigm,
    datasets=datasets,
    overwrite=True,
    suffix='demo',
)

# --------------------------
# Load data from MOABB
# --------------------------
# Note: get_data returns a dictionary with a key per subject.
data = paradigm.get_data(datasets[0])

In [20]:
# For illustration, if you have a single subject's data returned as a tuple:
X, y, metadata = data
print(f"X shape = {X.shape}, y shape = {y.shape}")


X shape = (1800, 3, 385), y shape = (1800,)


In [21]:
# Change y shape to one hot encoding
one_hot_y = []
i = 0
for label in y:
    if(y[i] == "feet"):
        one_hot_y.append(0)
    if(y[i] == "left_hand"):
        one_hot_y.append(1)
    if(y[i] == "right_hand"):
        one_hot_y.append(2)
    i+=1

one_hot_y = np.array(one_hot_y, dtype=np.int64)
X = np.array(X)

In [ ]:
y_pred = skorch_classifier.predict(X)

array([[1, 0, 1, ..., 1, 0, 1],
       [0, 1, 0, ..., 1, 0, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [0, 1, 0, ..., 1, 1, 0],
       [0, 1, 0, ..., 1, 1, 0],
       [1, 0, 1, ..., 1, 0, 0]])

In [23]:
y_pred.shape

(1800, 7)

In [24]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

class SkorchClassifierWrapper:
    def __init__(self, skorch_classifier):
        self.skorch_classifier = skorch_classifier
        
    def fit(self, X, y):
        # The classifier is already trained, so we just return self
        return self
        
    def predict(self, X):
        return self.skorch_classifier.predict(X)
    
    def predict_proba(self, X):
        return self.skorch_classifier.predict_proba(X)

# Create the pipeline
pipeline = Pipeline([
    ('classifier', SkorchClassifierWrapper(skorch_classifier))
])

In [25]:
results = evaluation.process(pipelines=dict(demo_pipeline=pipeline))

Zhou2016-WithinSession:   0%|          | 0/4 [00:00<?, ?it/s]

Reading 0 ... 305029  =      0.000 ...  1220.116 secs...


/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/Zhou2016.py:104: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = read_raw_cnt(fname, preload=True, eog=["VEOU", "VEOL"])
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/Zhou2016.py:104: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = read_raw_cnt(fname, preload=True, eog=["VEOU", "VEOL"])


Used Annotations descriptions: ['feet', 'left_hand', 'right_hand']
Reading 0 ... 430479  =      0.000 ...  1721.916 secs...


/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/Zhou2016.py:104: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = read_raw_cnt(fname, preload=True, eog=["VEOU", "VEOL"])
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/Zhou2016.py:104: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = read_raw_cnt(fname, preload=True, eog=["VEOU", "VEOL"])


...

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand']
Reading 0 ... 233249  =      0.000 ...   932.996 secs...
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand']
Reading 0 ... 226219  =      0.000 ...   904.876 secs...


/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/Zhou2016.py:104: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = read_raw_cnt(fname, preload=True, eog=["VEOU", "VEOL"])
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/Zhou2016.py:104: RuntimeWarning: Could not define the number of bytes automatically. Defaulting to 2.
  raw = read_raw_cnt(fname, preload=True, eog=["VEOU", "VEOL"])
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/Zhou2016.py:104: RuntimeWarning:   Could not parse meas date from the header. Setting to None.
  raw = read_raw_cnt(fname, preload=True, eog=["VEOU", "VEOL"])
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand']


/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 90 events (all good), 0 – 3 s (baseline off), ~1.6 MiB, data loaded,
 'left_hand': 30
 'right_hand': 30
 'feet': 30>
  warn(f"warnEpochs {epochs}")
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 89 events (all good), 0 – 3 s (baseline off), ~1.5 MiB, data loaded,
 'left_hand': 30
 'right_hand': 29
 'feet': 30>
  warn(f"warnEpochs {epochs}")
/Users/andresalvarezolmo/Documents/hume/ACS/playground/BCI-2023-Workshop_Learning-from-small-datasets/venv/lib/python3.10/site-packages/moabb/datasets/preprocessing.py:278: UserWarning: warnEpochs <Epochs | 75 events (all good), 0 – 3 s (baseline off), ~1.3 MiB, data loaded,
 'left_hand': 

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL braindecode.models.eegnet.EEGNetv4 was not an allowed global by default. Please use `torch.serialization.add_safe_globals([EEGNetv4])` or the `torch.serialization.safe_globals([EEGNetv4])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by default 